In [2]:
import sys
import os
import cobra
import cplex 
import libsbml
import pandas as pd
import copy
from pathlib import Path
import memote
import csv
import pytest


#Change working dir first, ty ChatGPT, much loves
cwd = os.getcwd()
# Split the path into a list of directories
directories = cwd.split(os.sep)
# Remove the last two directories from the list
directories = directories[:-2]
# Join the directories back into a path
new_cwd = os.sep.join(directories)
# Change the current working directory to the new path
os.chdir(new_cwd)

sys.path.append("./src")

import model_manipulation  as mm


In [3]:
#This jupyter notebook is to test out the basic properties of the final one-cell model, 
#based on Thiel & Fleming (2018?)
# https://opencobra.github.io/cobratoolbox/latest/tutorials/tutorialModelSanityChecks.html

model = cobra.io.read_sbml_model('./model/ios2164_1cell.xml')
model.solve = 'gurobi'

tol = 1e-6

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-09


In [ ]:
#This is to test the original iteration of iOS2164 made by Lakshmanan
model2 =cobra.io.read_sbml_model("./Supplementary_PrevLit/Lakshmanan et al (2015)/ios2164.xml")

#Test nga natin yung  kung ganyan rin siya.


for item in model2.reactions:
    if "EX_" in item.id:
        item.lower_bound = 0


model2.reactions.get_by_id('EX_photonVis_LPAREN_e_RPAREN_').lower_bound = -1000
#     model2.reactions.get_by_id('EX_o2_LPAREN_e_RPAREN_').lower_bound = -10
#     model2.reactions.get_by_id('EX_co2_LPAREN_e_RPAREN_').lower_bound = -10
model2.reactions.get_by_id('EX_fe3_LPAREN_e_RPAREN_').lower_bound = -10
# model2.reactions.get_by_id('EX_mg2_LPAREN_e_RPAREN_').lower_bound = -10
model2.reactions.get_by_id('EX_no3_LPAREN_e_RPAREN_').lower_bound = -10
model2.reactions.get_by_id('EX_so4_LPAREN_e_RPAREN_').lower_bound = -10
model2.reactions.get_by_id('EX_h2o_LPAREN_e_RPAREN_').lower_bound = -10
model2.reactions.get_by_id('EX_pi_LPAREN_e_RPAREN_').lower_bound = -1000
model2.reactions.get_by_id('EX_h_LPAREN_e_RPAREN_').lower_bound = -10


with model2:
    
#     cobra.flux_analysis.add_loopless(model2)
    summ = model2.optimize()
summ

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_ala_DASH_L_c "L-Alanine">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_ala_DASH_L_c "L-Alanine">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_arg_DASH_L_c "L-Arginine">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_arg_DASH_L_c "L-Arginine">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_asp_DASH_L_c "L-Aspartate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_asp_DASH_L_c "L-Aspartate">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_cys_D

In [ ]:
#Notes:
#Main paths from pi_e include feeding towards erroneous ATP generation. 
#ATP and cellulose (derived from another ATP generating process) also feeds into biomass equation
#pFBA doesn't work to remove the loops
#Loopless FBA (add_loopless) doesn't work


In [ ]:
#Notes for curated 1-cell model:
#Erroneous loops particularly in GAPDH1ys have abnormally high flux, which in turn generates more ATP and feeding to biomass

In [ ]:
#The original modellers for ios2164 turned off a number of reactions to prevent co-factor  recycling:
#From Lakshmanan et al (2016):

#Considering the large size of iOS2164, a set of reactions belonging to folates metabolism, 
#glycolysis/gluconeogenesis, nitrogen metabolism, and sulfate metabolism was also constrained 
#at zero in leaf and coleoptile simulations to avoid free cofactor recycling. 
#The complete sets of reactions that are inactivated during both the simulations are listed in Supplemental Data Set S2.


In [ ]:
#I'll try to export these reactions to a list first then implement it, then check if the solutions are still ok afterwards

Part 1: Checking for the reactions mentioned by Lakshmanan et al (2015) that need to be  turned off and check it in Fluxer
    - Cell 1: Load inactive reactions
    - Cell : Parametrize model and export for testing with Fluxer
    - Cell 2, 3, 4, 5: Generation of model to test with Fluxer, parametrized based on which reactions to turn off

In [ ]:
#Load inactive reactions in model
file = csv.reader(open('./misc/leaf_inactivated.tsv'), delimiter='\t')
leaf_inactive_rxns = list()
for rows in file:
    row = str()
    for rxns in rows:
        row += str(rxns)
    leaf_inactive_rxns.append(row)

#Actually applying this list prevents any biomass from actually forming so what I should do is that
#I'd just double check the model if it produces any loops
leaf_intact_bounds_dict = {}
for rxns in model.reactions:
    if rxns.id in leaf_inactive_rxns:
        leaf_intact_bounds_dict[rxns.id] = rxns.bounds
    

Next test: check if any of the reactions listed above produce any biomass after deactivating all of them at once. Checking will be done one-at-a-time

Model is parametrized w/ only phosphate and nh3 as input

Result:

All of the reactions above produce biomass even when turned off by themselves. Most likely the original modellers already took that to account and turned them off entirely. Hence, to facilitate my modelling I need to turn them all off simultaneously as part of my model inititialization.

However,  I do need to turn on some of the reactions, but I'll double check na lang rin.

Based on the comparisons above it seems all the reactions still produce erroneous cycling even if added in a stepwise fashion.



#1st attempt:  passed  complete  list from Lakshmanan  et  al (2016) to deactivate all co-factor
#Cycling related  genes. Was blocked as it included  PRISM_White_LED, which was the only wavelength
#that I've activated. Result was that solver
#was infeasible.

#2nd attempt: Passed  the  same  list as  above except for PRISM_White_LED. Result was that 
#the there  was a optimal solutiion to the solver. Furthermore, testing it with no phosphate input
#Returns 0 flux to Biomass.

#3rd  attempt: Same  conditions, no gas input. No production of biomass.


Other  results include the following:
- No production of ATP/biomass from other inorganic cofactors (h+, h2o, and others)
- No "leaks" since no fluxes are produced even when demand reactions are present 
- Should I test if biomass is produced if model is supplied with ATP? Let's try.


Part 2: Testing for Spurious Biomass/ATP formation
    Test 1: forward and backward ATP drain
    Test 2: forced ATP input
    Test 3: 

In [7]:
# #Test 1
#Check if there is flux towards ATPS if there are no input fluxes

with model:
    #Turn off all boundary reactions (Closed system)
    DM_atp = model.reactions.ATPSm
    for item in model.boundary:
        item.bounds = (0,0)
#     model.add_reaction(DM_atp)
    #Check forward (from cell to outside)
    DM_atp.bounds = (0, 1)
    DM_atp_fw = model.optimize()
    #Check backward (From outside to cell)
    
    DM_atp.bounds = (-1, 0)
    DM_atp_bw = model.optimize()
    
    print('ATP fw: ', DM_atp_fw.status, 'ATP bw: ', DM_atp_bw.status)
    print('Objective function value: ', DM_atp_fw.objective_value, 'Objective function value: ', 
          DM_atp_bw.objective_value)

ATP fw:  optimal ATP bw:  optimal
Objective function value:  0.0 Objective function value:  0.0


In [8]:
#Test 2: Forced ATP flux

with model:
    #Turn off all boundary reactions (Closed system)
    DM_atp = model.reactions.ATPSm
    for item in model.boundary:
        item.bounds = (0,0)
#     model.add_reaction(DM_atp)
    #Check forward (from cell to outside)
    DM_atp.bounds = (1, 1)
    DM_atp_fw = model.optimize()
    #Check backward (From outside to cell)
    
    DM_atp.bounds = (-1, -1)
    DM_atp_bw = model.optimize()
    
    
    print('ATP fw: ', DM_atp_fw.status, 'ATP bw: ', DM_atp_bw.status)
    print('Objective function value: ', DM_atp_fw.objective_value, 'Objective function value: ', 
          DM_atp_bw.objective_value)

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


ATP fw:  infeasible ATP bw:  infeasible
Objective function value:  None Objective function value:  None


In [9]:
#Test if Model can produce ATP only with gases 

with model:
    #Set Demand reaction for ATP Synthase
    DM_atp = model.reactions.ATPSm
    
    
    #Turn off all except phosphate flux
    photo_medium = mm.read_medium_csv('./misc/photo_medium.csv', model)
    photo_medium['EX_co2(e)'] = 1
    photo_medium['EX_o2(e)'] = 1
    photo_medium['EX_pi(e)'] = 0
    photo_medium['EX_no3(e)'] = 1
    photo_medium['EX_photonVis(e)'] = 1000
    model.medium =photo_medium 
    
    
#     for item in model.boundary:
#         item.bounds = (0,0)
#     model.add_reaction(DM_atp)
    #Check forward (from cell to outside)
    DM_atp.bounds = (1, 1)
    DM_atp_fw = model.optimize()
    #Check backward (From outside to cell)
    
    DM_atp.bounds = (-1, -1)
    DM_atp_bw = model.optimize()

    print('ATP fw: ', DM_atp_fw.status, 'ATP bw: ', DM_atp_bw.status)
    print('Objective function value (FW): ', DM_atp_fw.objective_value, 'Objective function value (BW): ', 
          DM_atp_bw.objective_value)

ATP fw:  optimal ATP bw:  infeasible
Objective function value (FW):  0.0 Objective function value (BW):  None


/home/rtsantos3/anaconda3/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Both Forward and Backward reactions do not produce any biomass, which is good

In [10]:
#Test 3: Check if model can produce biomass w/o phosphate

#First check if model can create biomass from scratch  or from inorganic solutes

#Parametrize the model first using phototrophic  medium
photo_medium = mm.read_medium_csv('./misc/photo_medium.csv', model)
photo_medium['EX_co2(e)'] = 1
photo_medium['EX_o2(e)'] = 1
photo_medium['EX_pi(e)'] = 0
photo_medium['EX_no3(e)'] = 1
photo_medium['EX_photonVis(e)'] = 1000

model.medium = photo_medium

photo_medium
model.summary()


Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


Above result shows that inputting NO3, CO2 and O2 doesn't produce any biomass by themselves, which is good.

For the following code blocks we will be testing which reactions are unbalanced in the final one-cell model. (UPDATE: Moved to "Checking Mass balance" script")


